In [2]:
sc

<SparkContext master=local[2] appName=PySparkShell>

In [3]:
file_char = "datasets/spark/Characters.txt"
file_book = "datasets/spark/Books.txt"
file_edge = "datasets/spark/Edges.txt"

In [4]:
chars = sc.textFile(file_char)
books = sc.textFile(file_book)
edges = sc.textFile(file_edge)

In [8]:
chars.take(10)

['Vertex 1: 24-HOUR MAN/EMMANUEL',
 'Vertex 2: 3-D MAN/CHARLES CHANDLER & HAROLD CHANDLER',
 'Vertex 3: 4-D MAN/MERCURIO',
 'Vertex 4: 8-BALL/',
 'Vertex 5: A',
 "Vertex 6: A'YIN",
 'Vertex 7: ABBOTT, JACK',
 'Vertex 8: ABCISSA',
 'Vertex 9: ABEL',
 'Vertex 10: ABOMINATION/EMIL BLONSKY']

In [9]:
books.take(10)

['Vertex 6487: AA2 35',
 'Vertex 6488: M/PRM 35',
 'Vertex 6489: M/PRM 36',
 'Vertex 6490: M/PRM 37',
 'Vertex 6491: WI? 9',
 'Vertex 6492: AVF 4',
 'Vertex 6493: AVF 5',
 'Vertex 6494: H2 251',
 'Vertex 6495: H2 252',
 'Vertex 6496: COC 1']

In [10]:
edges.take(10)

['*Vertices 19428 6486',
 '1 "24-HOUR MAN/EMMANUEL"',
 '2 "3-D MAN/CHARLES CHAN"',
 '3 "4-D MAN/MERCURIO"',
 '4 "8-BALL/"',
 '5 "A"',
 '6 "A\'YIN"',
 '7 "ABBOTT, JACK"',
 '8 "ABCISSA"',
 '9 "ABEL"']

In [11]:
def edges_filter(row):
    if '*' in row or '"' in row:
        return False
    else:
        return True
    
filtered_edge = edges.filter(edges_filter)
            

In [12]:
filtered_edge.take(5)

['1 6487',
 '2 6488 6489 6490 6491 6492 6493 6494 6495 6496',
 '3 6497 6498 6499 6500 6501 6502 6503 6504 6505',
 '4 6506 6507 6508',
 '5 6509 6510 6511']

In [13]:
character_book_map = filtered_edge.map(lambda x:x.split()).map(lambda x:(x[0], x[1:]))

In [15]:
character_book_map.take(5)

[('1', ['6487']),
 ('2',
  ['6488', '6489', '6490', '6491', '6492', '6493', '6494', '6495', '6496']),
 ('3',
  ['6497', '6498', '6499', '6500', '6501', '6502', '6503', '6504', '6505']),
 ('4', ['6506', '6507', '6508']),
 ('5', ['6509', '6510', '6511'])]

In [16]:
#character and no of occurence of it in books(same results can be achieved using mapValues)
character_book_map.map(lambda x:(x[0], len(x[1][0:]))).take(10)

[('1', 1),
 ('2', 9),
 ('3', 9),
 ('4', 3),
 ('5', 3),
 ('6', 4),
 ('7', 1),
 ('8', 2),
 ('9', 2),
 ('10', 15)]

In [17]:
def char_parse(row):
    row = row.split(":")
    return (row[0][7:], row[1].strip())

parsed_chars = chars.map(char_parse).collectAsMap()

In [20]:
[x for x in parsed_chars.items()][:5]

[('1', '24-HOUR MAN/EMMANUEL'),
 ('2', '3-D MAN/CHARLES CHANDLER & HAROLD CHANDLER'),
 ('3', '4-D MAN/MERCURIO'),
 ('4', '8-BALL/'),
 ('5', 'A')]

In [21]:
character_book_map.mapValues(lambda x:len(x)).map(lambda x:(parsed_chars[x[0]],x[1])).take(5)#get ids of characters and corresponding occurences in books

[('24-HOUR MAN/EMMANUEL', 1),
 ('3-D MAN/CHARLES CHANDLER & HAROLD CHANDLER', 9),
 ('4-D MAN/MERCURIO', 9),
 ('8-BALL/', 3),
 ('A', 3)]

In [22]:
character_book_map.mapValues(lambda x:len(x)).map(lambda x:(parsed_chars[x[0]],x[1])).reduceByKey(lambda x,y:x+y).take(5)

[('24-HOUR MAN/EMMANUEL', 1),
 ('3-D MAN/CHARLES CHANDLER & HAROLD CHANDLER', 9),
 ("A'YIN", 4),
 ('ABBOTT, JACK', 1),
 ('ABEL', 2)]

In [23]:
c = character_book_map.mapValues(lambda x:len(x)).map(lambda x:(parsed_chars[x[0]],x[1])).reduceByKey(lambda x,y:x+y).sortBy(lambda x:-x[1])

In [24]:
c.take(10) #top 10 most influencial characters

[('SPIDER-MAN/PETER PARKER', 1625),
 ('CAPTAIN AMERICA', 1367),
 ('IRON MAN/TONY STARK', 1168),
 ('THING/BENJAMIN J. GRIMM', 990),
 ('THOR/DR. DONALD BLAKE/SIGURD JARLSON II/JAKE OLSON/LOREN OLSON', 965),
 ('HUMAN TORCH/JOHNNY STORM', 908),
 ('MR. FANTASTIC/REED RICHARDS', 875),
 ('HULK/DR. ROBERT BRUCE BANNER', 841),
 ('WOLVERINE/LOGAN', 820),
 ('INVISIBLE WOMAN/SUE STORM RICHARDS', 782)]

In [25]:
import itertools

In [26]:
cooccurence_map =character_book_map.flatMap(lambda x:list(itertools.combinations(x[1], 2)))
                                                                               

In [27]:
cooccurence_map.map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y).take(5)

[(('6488', '6489'), 1),
 (('6488', '6491'), 1),
 (('6488', '6492'), 1),
 (('6488', '6494'), 1),
 (('6488', '6495'), 1)]